In [1]:
!pip install gradio sentence-transformers scikit-learn pytesseract pillow docx2txt pandas
!apt-get install -y tesseract-ocr


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 2 not upgraded.


In [2]:
import gradio as gr
import pandas as pd
import pytesseract
from PIL import Image
import docx2txt
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import tempfile
import os

# Load AI model
model = SentenceTransformer("all-MiniLM-L6-v2")

def extract_text(file_path):
    try:
        if file_path.endswith(".docx"):
            return docx2txt.process(file_path)

        elif file_path.endswith(".txt"):
            with open(file_path, "r", encoding="utf-8", errors="ignore") as f:
                return f.read()

        elif file_path.lower().endswith((".png", ".jpg", ".jpeg")):
            image = Image.open(file_path)
            return pytesseract.image_to_string(image)

    except:
        return ""

    return ""

def analyze_resumes(job_description, files, threshold):

    if not files:
        return "Please upload resumes.", None

    job_embedding = model.encode(job_description)
    results = []

    for file in files:
        text = extract_text(file.name)

        if not text or len(text.strip()) < 20:
            score = 0.0
        else:
            resume_embedding = model.encode(text)
            score = cosine_similarity(
                [resume_embedding], [job_embedding]
            )[0][0] * 100

        results.append({
            "Resume Name": os.path.basename(file.name),
            "Match Percentage": round(score, 2)
        })

    df = pd.DataFrame(results).sort_values(
        by="Match Percentage", ascending=False
    )

    selected = df[df["Match Percentage"] >= threshold]
    rejected = df[df["Match Percentage"] < threshold]

    summary = f"""
Total resumes processed: {len(df)}
Selected (≥{threshold}%): {len(selected)}
Rejected (<{threshold}%): {len(rejected)}
"""

    return summary, df

# Gradio UI
interface = gr.Interface(
    fn=analyze_resumes,
    inputs=[
        gr.Textbox(
            lines=5,
            label="Job Description",
            value="Looking for a Python developer with experience in Machine Learning, APIs, data analysis, and NLP."
        ),
        gr.File(file_count="multiple", label="Upload Resumes"),
        gr.Slider(0, 100, value=80, label="Match Threshold (%)")
    ],
    outputs=[
        gr.Textbox(label="Summary"),
        gr.Dataframe(label="Resume Match Results")
    ],
    title="🤖 AI Resume Screening System (Gradio)",
    description="OCR + Semantic AI based resume matcher. Supports scanned resumes, DOCX, and TXT files."
)

interface.launch(share=True)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://992c005d5f25a8666c.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
